In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Concatenate
from keras.optimizers import Adam
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Example time series data
data = pd.read_csv('your_time_series_data.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Feature engineering and preprocessing
def create_features(data):
    data['Month'] = data.index.month
    data['Quarter'] = data.index.quarter
    return data

data = create_features(data)

# Scaling features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data.drop('Target', axis=1))
X = scaled_features
y = data['Target'].values

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def build_gating_network(input_dim, num_experts):
    input_layer = Input(shape=(input_dim,))
    x = Dense(64, activation='relu')(input_layer)
    output_layer = Dense(num_experts, activation='softmax')(x)
    gating_model = Model(inputs=input_layer, outputs=output_layer)
    return gating_model

In [ ]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
# Train ARIMA Model (simplified example, usually you need to fine-tune the parameters)
arima_model = ARIMA(y_train, order=(5, 1, 0)).fit()

# Train Exponential Smoothing Model
exp_smoothing_model = ExponentialSmoothing(y_train).fit()

# Train Random Forest Regressor
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

# Train XGBoost Regressor
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

# Train SARIMA Model (simplified example)
sarima_model = SARIMAX(y_train, order=(1, 1, 1)).fit(disp=False)

# Train LSTM Model
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
lstm_model = build_lstm_model((X_train_reshaped.shape[1], X_train_reshaped.shape[2]))
lstm_model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, verbose=1)

In [ ]:
def get_expert_predictions(X, X_reshaped):
    # Predict using each model
    arima_pred = arima_model.forecast(steps=len(X))
    exp_smoothing_pred = exp_smoothing_model.forecast(steps=len(X))
    rf_pred = rf_model.predict(X)
    xgb_pred = xgb_model.predict(X)
    sarima_pred = sarima_model.forecast(steps=len(X))
    lstm_pred = lstm_model.predict(X_reshaped).flatten()

    # Stack predictions for input to the gating network
    expert_predictions = np.stack((arima_pred, exp_smoothing_pred, rf_pred, xgb_pred, sarima_pred, lstm_pred), axis=1)
    return expert_predictions

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
train_expert_preds = get_expert_predictions(X_train, X_train_reshaped)

In [ ]:
# Build the gating network
input_dim = X_train.shape[1]
num_experts = 6  # Number of experts
gating_network = build_gating_network(input_dim, num_experts)
gating_network.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train the gating network using expert predictions
gating_network.fit(X_train, train_expert_preds, epochs=50, batch_size=32, verbose=1)

In [ ]:
# Prepare test data
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
test_expert_preds = get_expert_predictions(X_test, X_test_reshaped)

# Predict gating weights for the test set
gating_weights = gating_network.predict(X_test)

# Calculate the final MoE predictions as a weighted sum of expert predictions
moe_predictions = np.sum(gating_weights * test_expert_preds, axis=1)

# Evaluate the MoE model
mse = np.mean((moe_predictions - y_test) ** 2)
print(f'MoE Model MSE: {mse:.4f}')